In [1]:
import torch
from torch import nn

In [2]:
import numpy as np

In [3]:
def check_gpu():
    is_cuda = torch.cuda.is_available()
    if is_cuda:
        device = torch.device("cuda")
        print("GPU is avaiavle.")
    else:
        device = torch.device("cpu")
        print("Using CPU, GPU is not available.")
    return device

In [4]:
device = check_gpu()

Using CPU, GPU is not available.


In [ ]:
class rNN_ntb(nn.Module):
    def __init__(self, in_dim,out_dim,hidden_dim,n_layers):
        super(rNN_ntb,self).init()